In [1]:
!pip install plaid-python dotenv


  Using cached plaid_python-37.1.0.tar.gz (1.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached nulltype-2.3.1-py2.py3-none-any.whl.metadata (10 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached nulltype-2.3.1-py2.py3-none-any.whl (11 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
  DEPRECATION: Building 'plaid-python' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'plaid-python'. Discussion c

In [ ]:
# Cell 2: Imports + Plaid client configuration
import os
from dotenv import load_dotenv
load_dotenv()

import time
import json
from datetime import date, timedelta

from plaid import ApiClient, Configuration, Environment
from plaid.api import plaid_api

from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.country_code import CountryCode
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.products import Products
from plaid.model.link_token_create_request_statements import LinkTokenCreateRequestStatements
from plaid.model.consumer_report_permissible_purpose import ConsumerReportPermissiblePurpose
from plaid.model.link_token_create_request_cra_options import LinkTokenCreateRequestCraOptions
from plaid.model.link_token_get_request import LinkTokenGetRequest
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.plaid_error import PlaidError
import plaid


# ==== CONFIG - FILL THESE IN ====
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")  # use sandbox secret
PLAID_ENV = Environment.Sandbox       # Sandbox for testing

# Create Plaid client
configuration = Configuration(
    host=PLAID_ENV,
    api_key={
        "clientId": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET,
    }
)
api_client = ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

print("Plaid client initialized in Sandbox ✅")


Plaid client initialized in Sandbox ✅


In [8]:
# Cell 3 (updated): Create a Link token configured for Hosted Link and print the hosted_link_url
from plaid.model.products import Products


PLAID_PRODUCTS = ['auth', 'transactions', 'signal']   # adjust as needed
products = [Products(p) for p in PLAID_PRODUCTS]

PLAID_COUNTRY_CODES = ['US', 'CA']
user_token = None  # if you use /user/create later, you can plug that in

LINK_TOKEN = None  # will be set below


def create_link_token():
    global LINK_TOKEN

    try:
        # Base Link token request
        link_token_request = LinkTokenCreateRequest(
            products=products,
            client_name="Plaid Colab Demo",  # <= max 30 chars
            country_codes=[CountryCode(c) for c in PLAID_COUNTRY_CODES],
            language="en",
            user=LinkTokenCreateRequestUser(
                client_user_id=str(time.time()),
                # These are optional for your current manual flow,
                # but good to have for when you later use SMS/email delivery.
                phone_number="+14155550123",
                email_address="sandbox-user@example.com",
            ),
        )

        # 🔹 This is the key: enable Hosted Link with an *empty* object
        # Your plaid-python version treats this as a generic object,
        # and Plaid will still return `hosted_link_url` in the response.
        link_token_request['hosted_link'] = {}

        # Optional: if you ever add 'statements' to PLAID_PRODUCTS
        if Products('statements') in products:
            statements = LinkTokenCreateRequestStatements(
                end_date=date.today(),
                start_date=date.today() - timedelta(days=30)
            )
            link_token_request['statements'] = statements

        # Optional: CRA options (only if you actually add CRA products)
        cra_products = ["cra_base_report", "cra_income_insights", "cra_partner_insights"]
        if any(p in cra_products for p in PLAID_PRODUCTS):
            link_token_request['user_token'] = user_token
            link_token_request['consumer_report_permissible_purpose'] = \
                ConsumerReportPermissiblePurpose('ACCOUNT_REVIEW_CREDIT')
            link_token_request['cra_options'] = LinkTokenCreateRequestCraOptions(
                days_requested=60
            )

        response = client.link_token_create(link_token_request)
        data = response.to_dict()

        LINK_TOKEN = data["link_token"]
        hosted_link_url = data.get("hosted_link_url")

        print("✅ Link token created")
        print("LINK_TOKEN:", LINK_TOKEN)
        print()
        print("👉 Open this URL in your browser to complete Hosted Link:")
        print(hosted_link_url)

        return data

    except plaid.ApiException as e:
        print("Plaid API exception while creating link token:")
        try:
            body = json.loads(e.body)
            print(json.dumps(body, indent=2))
        except Exception:
            print(e)
        raise


link_response = create_link_token()


✅ Link token created
LINK_TOKEN: link-sandbox-74bde010-38f9-4399-ab1f-9b9ddc6154f3

👉 Open this URL in your browser to complete Hosted Link:
https://secure.plaid.com/hl/ls29oqr56583s49844no6s4o4qqp1609s8


In [9]:
# Cell 4 (updated): After completing Hosted Link, get public_token and exchange it for access_token

from pprint import pprint  # nicer, no JSON serialization issues

def finalize_link_with_link_token(link_token: str):
    """
    1. /link/token/get -> get public_token (for Hosted Link / Delivery flows)
    2. /item/public_token/exchange -> get access_token
    """
    try:
        # Step 1: get info about Link session, including public_token (if ready)
        get_request = LinkTokenGetRequest(link_token=link_token)
        get_response = client.link_token_get(get_request)
        get_data = get_response.to_dict()

        print("🔍 /link/token/get response (Python dict):")
        pprint(get_data)   # <-- no json.dumps, avoids datetime issue

        public_token = get_data.get("public_token")
        if not public_token:
            print("\n❗ public_token is not available yet.")
            print("   Make sure you fully completed the Hosted Link flow in the browser.")
            return None

        print("\n✅ Got public_token:", public_token)

        # Step 2: exchange public_token for access_token
        exchange_request = ItemPublicTokenExchangeRequest(public_token=public_token)
        exchange_response = client.item_public_token_exchange(exchange_request)
        exchange_data = exchange_response.to_dict()

        access_token = exchange_data["access_token"]
        item_id = exchange_data["item_id"]

        print("\n🎉 SUCCESS")
        print("access_token:", access_token)
        print("item_id:", item_id)

        # In a real app: store access_token securely (DB/secret store)
        return {
            "access_token": access_token,
            "item_id": item_id,
            "exchange_raw": exchange_data,
        }

    except plaid.ApiException as e:
        print("Plaid API exception while finalizing link:")
        try:
            error_body = json.loads(e.body)
            print(json.dumps(error_body, indent=2))
        except Exception:
            print(e)
        return None


# Use the LINK_TOKEN from Cell 3
if LINK_TOKEN is None:
    print("LINK_TOKEN is None – run Cell 3 first.")
else:
    result = finalize_link_with_link_token(LINK_TOKEN)


🔍 /link/token/get response (Python dict):
{'created_at': datetime.datetime(2025, 12, 5, 0, 29, 2, tzinfo=tzutc()),
 'expiration': datetime.datetime(2025, 12, 5, 0, 59, 2, tzinfo=tzutc()),
 'link_sessions': [{'events': [{'event_id': 'd71fa613-62f8-4f3e-a203-380d54745d7a',
                                'event_metadata': {'institution_id': 'ins_19',
                                                   'institution_name': 'Regions '
                                                                       'Bank',
                                                   'request_id': 'RfVz82xq1ZW0Fpv'},
                                'event_name': 'HANDOFF',
                                'timestamp': '2025-12-05T00:29:40Z'},
                               {'event_id': '730b7f48-343d-4c30-a11c-d75d39a82a06',
                                'event_metadata': {'institution_id': 'ins_19',
                                                   'institution_name': 'Regions '
                               

In [10]:
# Cell 4 (updated): After completing Hosted Link, get public_token and exchange it for access_token
# 1. /link/token/get -> get public_token (for Hosted Link / Delivery flows)
# Step 1: get info about Link session, including public_token (if ready)

from pprint import pprint

get_request = LinkTokenGetRequest(link_token=LINK_TOKEN)
get_response = client.link_token_get(get_request)
get_data = get_response.to_dict()

print("🔍 /link/token/get response (Python dict):")
pprint(get_data)   # <-- no json.dumps, avoids datetime issue

if "public_token" in get_data and get_data["public_token"]:
    PUBLIC_TOKEN = get_data["public_token"]

# 2) Hosted Link pattern
for session in get_data.get("link_sessions", []):
    results = session.get("results", {})
    for item_result in results.get("item_add_results", []):
        public_token = item_result.get("public_token")
        if public_token:
            PUBLIC_TOKEN = public_token

if not PUBLIC_TOKEN:
    print("\n❗ public_token is not available yet.")
    print("   Make sure you fully completed the Hosted Link flow in the browser.")


print("\n✅ Got public_token:", public_token)


🔍 /link/token/get response (Python dict):
{'created_at': datetime.datetime(2025, 12, 5, 0, 29, 2, tzinfo=tzutc()),
 'expiration': datetime.datetime(2025, 12, 5, 0, 59, 2, tzinfo=tzutc()),
 'link_sessions': [{'events': [{'event_id': 'd71fa613-62f8-4f3e-a203-380d54745d7a',
                                'event_metadata': {'institution_id': 'ins_19',
                                                   'institution_name': 'Regions '
                                                                       'Bank',
                                                   'request_id': 'RfVz82xq1ZW0Fpv'},
                                'event_name': 'HANDOFF',
                                'timestamp': '2025-12-05T00:29:40Z'},
                               {'event_id': '730b7f48-343d-4c30-a11c-d75d39a82a06',
                                'event_metadata': {'institution_id': 'ins_19',
                                                   'institution_name': 'Regions '
                               

In [12]:
# Cell 4 (updated): After completing Hosted Link, get public_token and exchange it for access_token

from pprint import pprint  # nicer, no JSON serialization issues
try:
    # Step 2: exchange public_token for access_token
    exchange_request = ItemPublicTokenExchangeRequest(public_token=PUBLIC_TOKEN)
    exchange_response = client.item_public_token_exchange(exchange_request)
    exchange_data = exchange_response.to_dict()

    access_token = exchange_data["access_token"]
    item_id = exchange_data["item_id"]

    print("\n🎉 SUCCESS")
    # print("access_token:", access_token)
    # print("item_id:", item_id)

    # In a real app: store access_token securely (DB/secret store)
    access_credentials = {
        "access_token": access_token,
        "item_id": item_id,
        "exchange_raw": exchange_data,
    }

except plaid.ApiException as e:
    print("Plaid API exception while finalizing link:")
    try:
        error_body = json.loads(e.body)
        print(json.dumps(error_body, indent=2))
    except Exception:
        print(e)

# print(access_credentials)


🎉 SUCCESS


In [13]:
import plaid
from plaid.model.transactions_sync_request import TransactionsSyncRequest
import datetime

request = TransactionsSyncRequest(
    access_token=access_credentials['access_token'],
)
response = client.transactions_sync(request)
transactions = response['added']

# the transactions in the response are paginated, so make multiple calls while incrementing the cursor to
# retrieve all transactions
while (response['has_more']):
    request = TransactionsSyncRequest(
        access_token=access_token,
        cursor=response['next_cursor']
    )
    response = client.transactions_sync(request)
    transactions += response['added']

transactions[:5]

[{'account_id': 'pwbp1QozZ4F6m5nAgd3MsQD5Am1ABGUpedoRd',
  'account_owner': None,
  'amount': 6.33,
  'authorized_date': datetime.date(2025, 11, 27),
  'authorized_datetime': None,
  'category': None,
  'category_id': None,
  'check_number': None,
  'counterparties': [{'confidence_level': 'VERY_HIGH',
                      'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd',
                      'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png',
                      'name': 'Uber',
                      'phone_number': None,
                      'type': 'merchant',
                      'website': 'uber.com'}],
  'date': datetime.date(2025, 11, 28),
  'datetime': None,
  'iso_currency_code': 'USD',
  'location': {'address': None,
               'city': None,
               'country': None,
               'lat': None,
               'lon': None,
               'postal_code': None,
               'region': None,
               'store_number': None},
  'logo_url': 'ht

In [14]:
# import plaid
from plaid.model.transactions_sync_request import TransactionsSyncRequest

# 1. Initial sync with no cursor
request = TransactionsSyncRequest(
    access_token=access_credentials["access_token"],
)

response = client.transactions_sync(request)

all_added = response["added"]
all_modified = response["modified"]
all_removed = response["removed"]
cursor = response["next_cursor"]

# 2. Continue until has_more = False
while response["has_more"]:
    request = TransactionsSyncRequest(
        access_token=access_credentials["access_token"],
        cursor=cursor
    )
    response = client.transactions_sync(request)

    all_added += response["added"]
    all_modified += response["modified"]
    all_removed += response["removed"]

    cursor = response["next_cursor"]

# 3. Display sample
all_added[:5]


[{'account_id': 'pwbp1QozZ4F6m5nAgd3MsQD5Am1ABGUpedoRd',
  'account_owner': None,
  'amount': 6.33,
  'authorized_date': datetime.date(2025, 11, 27),
  'authorized_datetime': None,
  'category': None,
  'category_id': None,
  'check_number': None,
  'counterparties': [{'confidence_level': 'VERY_HIGH',
                      'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd',
                      'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png',
                      'name': 'Uber',
                      'phone_number': None,
                      'type': 'merchant',
                      'website': 'uber.com'}],
  'date': datetime.date(2025, 11, 28),
  'datetime': None,
  'iso_currency_code': 'USD',
  'location': {'address': None,
               'city': None,
               'country': None,
               'lat': None,
               'lon': None,
               'postal_code': None,
               'region': None,
               'store_number': None},
  'logo_url': 'ht

In [ ]:
import plaid
from plaid.model.transactions_sync_request import TransactionsSyncRequest

def sync_transactions(access_token):
    request = TransactionsSyncRequest(access_token=access_token)
    response = client.transactions_sync(request)

    all_added = response["added"]
    all_modified = response["modified"]
    all_removed = response["removed"]
    cursor = response["next_cursor"]

    while response["has_more"]:
        request = TransactionsSyncRequest(
            access_token=access_token,
            cursor=cursor
        )
        response = client.transactions_sync(request)

        all_added += response["added"]
        all_modified += response["modified"]
        all_removed += response["removed"]
        cursor = response["next_cursor"]

    return {
        "added": all_added,
        "modified": all_modified,
        "removed": all_removed,
        "cursor": cursor
    }

# --- RUN IT ---
result = sync_transactions(access_credentials["access_token"])

print("Added:", len(result["added"]))
print("Modified:", len(result["modified"]))
print("Removed:", len(result["removed"]))
print("Cursor:", result["cursor"])

result["added"][:5]
